# Surprise homework
Because all labels in this document are in English and this task is about predictions I have predicted that it should be done in English.

In [1]:
from collections import defaultdict
import pandas as pd
from surprise import Dataset
from surprise import NormalPredictor
from surprise import KNNWithMeans
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold

### Functions

There two functions exectly from surprise FAQ ([top-N recommendation](https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-get-the-top-n-recommendations-for-each-user) and [precision@k and recall@k](https://surprise.readthedocs.io/en/stable/FAQ.html#how-to-get-the-top-n-recommendations-for-each-user)). It looks like copypaste, but in fact it is the copypaste with subsequent *awareness* and *understanding the code*.It makes no sense to change the code, because the developers have made it rather scalable and flexible. Also I have written one own function to serialize the resulting list of movies into the required text format.

In [2]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

def serialize(uid, movies):
    """Return serialized recommendation data for the user"""
    
    result = f'User {uid}\n'

    id_len = 0
    content_len = 0

    for m in movies:
        id_len = max(id_len, len(m['id']))
        content_len = max(content_len, len(m['title'] + m['release']) + 8)

    for m in movies:
        result += (f"   {m['id'].ljust(id_len)} " +
            f"('{m['title']}', '{m['release']}')".ljust(content_len) +
            f" {m['rating']}\n")
    
    return result

### Load data

In [3]:
data = Dataset.load_builtin('ml-100k')

# Number of future folds for the data set
N_FOLDS = 5

# My variant (I hope)
USER_ID = 22

# DataFrame for all items with columns 'title' for title of movie and 'release' for release date of this movie
items_df = pd.read_csv(
    'u.item',
    sep='|',
    encoding='ISO-8859-1',
    header=None,
    usecols=[1, 2],
    names=['title', 'release']
)

### Define algorithms

In [4]:
# Dictionary of the algorithms. Key is a title of the algorithm and the value is surprise object of it
algs = {}

# To predict a random rating on the distribution of all ratings in a set we have to calculate normal distribution
algs['Predicting a random rating based on the distribution of all ratings in a set'] = NormalPredictor()

# Returns kNN algorithm with k = 30 and required metric 
get_kNN = lambda metric: KNNWithMeans(k=30, sim_options={'name': metric})

algs['User-based collaborative filtering, kNN method, k = 30, cosine metric'] = get_kNN('cosine')

algs['User-based collaborative filtering, kNN method, k = 30, Mean Squared Difference metric'] = get_kNN('msd')

algs['User-based collaborative filtering, kNN method, k = 30, Pearson correlation metric'] = get_kNN('pearson')

algs['SVD algorithm'] = SVD()

### Select algorithm

In [5]:
# This dictionary is similar to the dictionary of algorithms. Key is a title of the algorithm and the value is its root mean square error
RMSEs = {}

# Highlighting text in color and bold (or just color if you are a fan of Windows console) is set up just for fun
green = lambda str: f'\x1b[1;32m{str}\x1b[0m'
cyan = lambda str: f'\x1b[1;36m{str}\x1b[0m'

# Commit the cross validations with visualization (verbose option) and fill the RMSEs dictionary
for title, alg in algs.items():
    print(cyan(f'{title}:'))
    cv_result = cross_validate(alg, data, measures=['rmse'], cv=N_FOLDS, verbose=True)
    RMSEs[title] = cv_result['test_rmse'].mean()
    print()

best_alg = min(RMSEs, key=RMSEs.get)
print(f'In my humble opinion the best algorithm is {green(best_alg)} '
    f'with RMSE equal to {RMSEs[best_alg]}.')
best_alg = algs[best_alg]

Predicting a random rating based on the distribution of all ratings in a set:
Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5095  1.5337  1.5176  1.5214  1.5191  1.5203  0.0078  
Fit time          0.22    0.24    0.23    0.20    0.68    0.31    0.18    
Test time         0.19    0.17    0.29    0.23    0.60    0.30    0.16    

User-based collaborative filtering, kNN method, k = 30, cosine metric:
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  F

### Calculate precision@k and recall@k
Here we have two ways. The first is to calculate these metrics for all items. And the second is to split the dataset into several parts, calculate the metrics for each of them, and then calculate the average. I have implemented a split option exectly like I had done when have been calculating the RMSE. But if you wanted to go the first way, you would have to write:

    from surprise.model_selection import train_test_split
    trainset, testset = train_test_split(data, 1 / N_FOLDS)

And then just copy, with slight obvious changes, the body of the loop below.

In [6]:
# Dictionary mean 
dict_mean = lambda d: sum(x for x in d.values()) / len(d)

# Sums of presision and recall. They are needed to solve averages.
precisions_sum = 0
recalls_sum = 0

# Cross-validation iterator
kf = KFold(n_splits=N_FOLDS)

for trainset, testset in kf.split(data):
    predictions = best_alg.fit(trainset).test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=5, threshold=3.52)
    precisions_sum += dict_mean(precisions)
    recalls_sum += dict_mean(recalls)

print('For this algorithm:')
print(f'{green("precision@k")} is {precisions_sum / N_FOLDS}')
print(f'{green("recall@k   ")} is {recalls_sum / N_FOLDS}')

For this algorithm:
precision@k is 0.7237053537323097
recall@k    is 0.40599616522041015


### Predict

In [7]:
# We will use full train set for the best test
trainset = data.build_full_trainset()

# All these data are not in the trainset
testset = trainset.build_anti_testset()

# Training and predicting
predictions = best_alg.fit(trainset).test(testset)

# There are NO movies in this top, which user could see, unless they would not be in testset
top_for_user = get_top_n(predictions, 5)[str(USER_ID)]

recomendations = []
for id, rating in top_for_user:
    movie = dict(items_df.iloc[int(id)])
    # As I understood from the example, the resulting ID starts with one
    movie['id'] = str(int(id) + 1)
    movie['rating'] = str(round(rating, 3))
    recomendations.append(movie)

### Print and save to file

In [8]:
# Serialized recomendations
result = serialize(USER_ID, recomendations)

print(result)

with open('result.txt', 'w') as f:
    f.write(result)

User 22
   101 ('Heavy Metal (1981)', '08-Mar-1981')                          4.885
   184 ('Army of Darkness (1993)', '01-Jan-1993')                     4.781
   13  ('Mighty Aphrodite (1995)', '30-Oct-1995')                     4.766
   314 ('3 Ninjas: High Noon At Mega Mountain (1998)', '01-Jan-1997') 4.749
   170 ('Cinema Paradiso (1988)', '01-Jan-1988')                      4.715

